In [41]:
# Dependencies and Setup
import json
import pandas as pd
import requests
import numpy as py
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine, inspect, func, distinct
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3
from sqlite3 import Error
import datetime as DT

In [3]:
#Bring in data from internet as object then convert ot Json
player_response_obj=requests.get("http://data.nba.net/prod/v1/2018/players.json")
player_response=player_response_obj.json()

In [4]:
# Count # of Players
player_response_count=player_response["league"]["standard"]
player_count=len(player_response_count)
player_count

499

In [5]:
#Bring in elements for NBA_Player Table
NBA_Players=[]

for i in range (0,player_count-1):
    player_response_list=player_response["league"]["standard"][i]
    firstName=player_response_list["firstName"]
    lastName=player_response_list["lastName"]
    personId=player_response_list["personId"]
    teamId=player_response_list["teamId"]
    heightFeet=player_response_list["heightFeet"]
    heightInches=player_response_list["heightInches"]
    weightPounds=player_response_list["weightPounds"]
    dateOfBirthUTC=player_response_list["dateOfBirthUTC"]
    collegeName=player_response_list["collegeName"]

    # Create an object list to store NBA Player data as an object
   
    NBA_Players.append({"firstName":firstName,
                       "lastName":lastName,
                       "personId":personId,
                       "teamId":teamId,
                       "heightFeet":heightFeet,
                       "heightInches":heightInches,
                       "weightPounds":weightPounds,
                       "dateOfBirth":dateOfBirthUTC,
                       "collegeName":collegeName})

    #Test Object list
    NBA_Players

In [53]:
NBA_Player_df_initial=pd.DataFrame(NBA_Players)
NBA_Player_df_initial=NBA_Player_df_initial[(NBA_Player_df_initial["teamId"] != '')]
                                      
NBA_Player_df=NBA_Player_df_initial[["personId","lastName","firstName","teamId",
                                    "heightFeet","heightInches","weightPounds",
                                    "dateOfBirth","collegeName"]]

NBA_Player_df["personId"]=NBA_Player_df["personId"].astype(int)
NBA_Player_df["teamId"]=NBA_Player_df["teamId"].astype(int)                        
NBA_Player_df["lastName"]=NBA_Player_df["lastName"].astype(str)
NBA_Player_df["firstName"]=NBA_Player_df["firstName"].astype(str)
NBA_Player_df["heightFeet"]=NBA_Player_df["heightFeet"].astype(int)
NBA_Player_df["heightInches"]=NBA_Player_df["heightInches"].astype(int)
NBA_Player_df["weightPounds"]=NBA_Player_df["weightPounds"].astype(int)
NBA_Player_df["collegeName"]=NBA_Player_df["collegeName"].astype(str)
NBA_Player_df['dateOfBirth'] = pd.to_datetime(NBA_Player_df['dateOfBirth'])

NBA_Player_df.dtypes

personId                 int32
lastName                object
firstName               object
teamId                   int32
heightFeet               int32
heightInches             int32
weightPounds             int32
dateOfBirth     datetime64[ns]
collegeName             object
dtype: object

In [54]:
#Add Player Age
now = pd.Timestamp(DT.datetime.now())
NBA_Player_df['dob'] = pd.to_datetime(NBA_Player_df['dateOfBirth'], format='%m%d%y')    # 1
NBA_Player_df['dob'] = NBA_Player_df['dateOfBirth'].where(NBA_Player_df['dateOfBirth'] < now, NBA_Player_df['dateOfBirth'] -  py.timedelta64(100, 'Y'))   # 2
NBA_Player_df['age'] = (now - NBA_Player_df['dateOfBirth']).astype('<m8[Y]') 

In [55]:
NBA_Player_df=NBA_Player_df[["personId","lastName","firstName","teamId",
                                    "heightFeet","heightInches","weightPounds",
                                    "dateOfBirth","age","collegeName"]]

In [57]:
NBA_Player_df.dtypes

lastName                object
firstName               object
teamId                   int32
heightFeet               int32
heightInches             int32
weightPounds             int32
dateOfBirth     datetime64[ns]
age                    float64
collegeName             object
dtype: object

In [56]:
engine = create_engine("sqlite:///db/NBA_Data.sqlite")
conn = engine.connect()

engine.table_names()

NBA_Player_df.set_index("personId", inplace=True)

#Load Players_Info table
NBA_Player_df.to_sql('Players_Info',conn, if_exists='replace', index=True)